In [1]:
import pennylane as qml
from pennylane import numpy as np
import qemzmsepcV2 as qem

In [2]:
n_qubits = 4
dev = qml.device('default.mixed', wires=n_qubits)
nqubitschannel = qem.NqubitsChannel(n_qubits)
pauli_channel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.5)
qemzmsepc = qem.QEMZMSEPC(n_qubits)
p = 0.8

def trotterstep1(rotation_angle_of_rx, rotation_angle_of_rz):
    qml.RX(rotation_angle_of_rx, wires=0)
    qml.RX(rotation_angle_of_rx, wires=1)
    qml.RX(rotation_angle_of_rx, wires=2)
    qml.RX(rotation_angle_of_rx, wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.RZ(rotation_angle_of_rz, wires=1)
    qml.RZ(rotation_angle_of_rz, wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[1, 2])
    qml.RZ(rotation_angle_of_rz, wires=2)
    qml.CNOT(wires=[1, 2])

rotation_angles = [0, 0.5, 1, 1.5, 2, 2.5]
rotation_angle_of_rz = -0.2
for rotation_angle_of_rx in rotation_angles:
    ideal_output_func = qemzmsepc.noise_circuit(trotterstep1)
    z_ideal = qml.QNode(ideal_output_func, dev)(rotation_angle_of_rx, rotation_angle_of_rz)
    print(f"z_ideal is {z_ideal}")
    noise_output_func = qemzmsepc.noise_circuit(trotterstep1, p=p, kraus_matrices_of_a_pauli_channel=pauli_channel,
                                            need_gate_noise=True, need_measurement_noise=True)
    z_unmitigated = qml.QNode(noise_output_func, dev)(rotation_angle_of_rx, rotation_angle_of_rz)
    print(f"z_unmitigated is {z_unmitigated}")
    z_mitigated, _  = qemzmsepc.qemzmsepc(trotterstep1, p, dev, pauli_channel,
                                      rotation_angle_of_rx, rotation_angle_of_rz)
    print(f"z_mitigated is {z_mitigated}")
    print("------------------")

z_ideal is 1.0
z_unmitigated is 0.28191171312999513
z_mitigated is 1.0000000000000058
------------------
z_ideal is 0.5931327983656769
z_unmitigated is 0.16721108330085613
z_mitigated is 0.593132798365679
------------------
z_ideal is 0.08522112911847729
z_unmitigated is 0.024024834504663414
z_mitigated is 0.08522112911848502
------------------
z_ideal is 2.5037531073114627e-05
z_unmitigated is 7.058373277309127e-06
z_mitigated is 2.5037531073066387e-05
------------------
z_ideal is 0.029990685342117292
z_unmitigated is 0.008454725482737787
z_mitigated is 0.02999068534211013
------------------
z_ideal is 0.4119471515970569
z_unmitigated is 0.11613272722574977
z_mitigated is 0.4119471515970639
------------------
